In [1]:
! pip install lightgbm==3.3.5

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import lightgbm as lgb

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
dataset=pd.read_csv("insurance_pre.csv")

In [5]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [6]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [7]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [8]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [9]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]

In [10]:
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [11]:
dependent=dataset[['charges']]

In [12]:
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [46]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
param_grid = {'boosting type': ['gbdt'], 'num_leaves' : [31], 'max_depth' : [-1], 'learning_rate' : [0.1], 'n_estimators' : [100]}  

In [47]:
grid = GridSearchCV(LGBMRegressor(), param_grid, refit=True, verbose=3, n_jobs=-1)
grid.fit(independent,dependent)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[LightGBM] [Warning] Unknown parameter: type
[LightGBM] [Warning] boosting is set=, boosting_type=gbdt will be ignored. Current value: boosting=


GridSearchCV(estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'boosting type': ['gbdt'], 'learning_rate': [0.1],
                         'max_depth': [-1], 'n_estimators': [100],
                         'num_leaves': [31]},
             verbose=3)

In [51]:
re=grid.cv_results_
print("The R_score value for the best parameter{}:".format(grid.best_params_))

The R_score value for the best parameter{'boosting type': 'gbdt', 'learning_rate': 0.1, 'max_depth': -1, 'n_estimators': 100, 'num_leaves': 31}:


In [52]:
table=pd.DataFrame.from_dict(re)

In [53]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_boosting type,param_learning_rate,param_max_depth,param_n_estimators,param_num_leaves,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.993832,0.255884,0.027526,0.027756,gbdt,0.1,-1,100,31,"{'boosting type': 'gbdt', 'learning_rate': 0.1...",0.857835,0.776793,0.868698,0.831484,0.849952,0.836953,0.03244,1


In [54]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 0
BMI: 0
Children: 0
Sex Male 0 or 1: 0
Smoker Yes 0 or 1: 0


In [55]:
Future_Prediction=grid.predict([[age_input, bmi_input, children_input, sex_male_input, smoker_yes_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[2435.55288984]


In [56]:
import pickle
filename="finalized_model_LgboostRegressor_GridSearchCV.sav"
pickle.dump(grid,open(filename, 'wb'))

In [59]:
loaded_model=pickle.load(open("finalized_model_LgboostRegressor_GridSearchCV.sav", 'rb'))
result=loaded_model.predict([[0,0,0,0,0]])

In [60]:
result

array([2435.55288984])